In [27]:
import gym
import numpy as np
from stable_baselines3 import A2C, DDPG, SAC, PPO
from matplotlib import axis
import numpy as np
from floris.tools import FlorisInterface
from floris.tools.visualization import visualize_cut_plane
import matplotlib.pyplot as plt
from floris.tools.optimization.yaw_optimization.yaw_optimizer_sr import YawOptimizationSR

from gym import spaces

from stable_baselines3.common.env_checker import check_env
import os
import time
from tqdm import tqdm_notebook
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback

from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed


In [2]:
from matplotlib.pyplot import figure

#loading optimization package:
from scipy.optimize import minimize

from floris.tools.optimization.yaw_optimization.yaw_optimizer_scipy import (
    YawOptimizationScipy
)

from time import perf_counter as timerpc

import yaml
import math
from scipy import interpolate

In [41]:
test = FlorisInterface("gch.yaml")  
test_dict = test.floris.as_dict()

In [44]:
test_dict["flow_field"]["air_density"]

1.225

In [50]:
class CustomEnv(gym.Env):
    """
    The inputs are:
    t_steps     = number of timesteps pr simulation
    dist        ª= rotor diameters between the turbines
    nx          = number of turbines along x axis
    ny          = number of turbines along y axis
    turb_type   = The type of turbine used for the environment
    combination = The combination model
    deflection  = The deflection model
    turbulence  = The turbulence model
    velocity    = The wake velocity model
    VS_min      = minimum wind speed [m/s]
    VS_max      = maximum wind speed [m/s]
    TI_min      = minimum turbulence intensity
    TI_max      = maximum turbulence intensity
    wd_min      = minimum wind direction
    wd_max      = maximum wind direction
    yaw_max     = Is the maximum yaw offset allowed in degrees.
    
    """
    #Custom Environment that follows gym interface
    metadata = {'render.modes': ['human']}

    def __init__(self, t_steps = 10, dist = 5, nx = 3, ny = 3,
               turb_type = 'nrel_5MW', combination = 'sosfs', deflection = 'gauss',
               turbulence = 'crespo_hernandez', velocity = 'gauss',
               VS_min = 4, VS_max = 20, TI_min = 0.01, TI_max = 0.15, wd_min = 270, wd_max = 360, 
               yaw_max = 25, rho = 1.225):
        super(CustomEnv, self).__init__()
        
        self.t_max = t_steps   #The number of "simulations" pr episode.
        self.wind_speed_min = VS_min
        self.wind_speed_max = VS_max
        self.TI_min         = TI_min
        self.TI_max         = TI_max
        self.wd_min         = wd_min
        self.wd_max         = wd_max
        self.n_turb         = nx * ny
        self.yaw_max        = yaw_max
        self.rho            = rho
                     
        #Creates the base for the farm
        fi = FlorisInterface("gch.yaml")   
        
        #Turns it into a dictionary and then does the changes to the model
        fi_dict = fi.floris.as_dict()
        
        fi_dict["farm"]["turbine_type"] = [turb_type]
        fi_dict["wake"]["model_strings"]["combination_model"] = combination
        fi_dict["wake"]["model_strings"]["deflection_model"]  = deflection
        fi_dict["wake"]["model_strings"]["turbulence_model"]  = turbulence
        fi_dict["wake"]["model_strings"]["velocity_model"]    = velocity
        fi_dict["flow_field"]["air_density"]                  = rho
        
        # Turns it back into a floris object:
        self.fi = FlorisInterface(fi_dict)

        D = self.fi.floris.farm.rotor_diameters[0]

        x = np.linspace(0, D*dist*nx, nx)
        y = np.linspace(0, D*dist*ny, ny)

        xv, yv = np.meshgrid(x, y, indexing='xy')
        
        self.layout_x =  xv.flatten()
        self.layout_y = yv.flatten()
        
        #Reads and saves the power curve for one turbine:
        
        with open(turb_type+".yaml", 'r') as stream:
            try:
                parsed_yaml=yaml.safe_load(stream)
                #print(parsed_yaml)
            except yaml.YAMLError as exc:
                print(exc)

        ws_curve = parsed_yaml["power_thrust_table"]["wind_speed"]
        power_curve = parsed_yaml["power_thrust_table"]["power"]
        
        self.A = 3.14 * (D/2)**2
        self.power_curve = interpolate.interp1d(ws_curve, power_curve)
        

        # Define action and observation space
        
        # The actionspace is the 9 yaw angles.
        self.action_space = spaces.Box(low=-1, high=1,
                                            shape=(nx*ny,), dtype=np.float32)
        
        # The observationspace is WS, TI, WD:
        high = np.array([self.wind_speed_max, self.TI_max, self.wd_max], dtype = np.float32)
        low = np.array([self.wind_speed_min, self.TI_min, self.wd_min], dtype = np.float32)
        
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        self.reset()
        
    def step(self, action):
        #print("we did a step")
        if self.time > self.t_max:
            done =  True
        else:
            self.time += 1
            done = False
            
        self.fi.calculate_wake(yaw_angles=np.array([[action]]))  #weird format, but it's okay
        
        power_farm = self.fi.get_farm_power()
        
        rew = self.fi.get_farm_power()/self.power_ideal_farm
        
        reward = rew[0][0]   #We do this to only return a scalar insted of a array. It maybe doenst like that
            
        info = {}
        observation = np.array([self.ws, self.TI, self.wd], dtype = np.float32)
        return observation, reward, done, info
    
    def reset(self):
        
        #
        self.ws = np.random.uniform(self.wind_speed_min, self.wind_speed_max)
        self.wd = np.random.uniform(self.wd_min, self.wd_max)
        self.TI = np.random.uniform(self.TI_min, self.TI_max)
        
        #calculate the power for the ideal farm. Used for normalization
        self.power_ideal_farm = self.power_curve(self.ws)*self.n_turb* (1/2) * self.rho * self.A * self.ws**3
        
        self.fi.reinitialize(
            layout=(self.layout_x, self.layout_y),
            wind_directions=[self.wd],
            turbulence_intensity= self.TI,
            wind_speeds=[self.ws]
            )
        self.time = 0
        done = False
        
        observation = np.array([self.ws, self.TI, self.wd], dtype = np.float32)
        return observation  # reward, done, info can't be included
    
    def render(self, mode='human'):
        pass
    
    def close (self):
        pass

In [51]:
env = CustomEnv()

In [52]:
check_env(env)

In [53]:
model = PPO("MlpPolicy", env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [54]:
models_dir = "models/PPO"
log_dir = "logs"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [94]:
TIMESTEPS = 10_000

for i in range(1,30):
    model.learn(total_timesteps = TIMESTEPS, reset_num_timesteps = False, tb_log_name="PPO")
    model.save(f"{models_dir}/{TIMESTEPS*i}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12       |
|    ep_rew_mean     | 11.7     |
| time/              |          |
|    fps             | 76       |
|    iterations      | 1        |
|    time_elapsed    | 26       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 12           |
|    ep_rew_mean          | 11.6         |
| time/                   |              |
|    fps                  | 76           |
|    iterations           | 2            |
|    time_elapsed         | 53           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0066753617 |
|    clip_fraction        | 0.0714       |
|    clip_range           | 0.2          |
|    entropy_loss         | -12.8        |
|    explained_variance   | 0.000694     |
|    learning_r

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 12           |
|    ep_rew_mean          | 11.5         |
| time/                   |              |
|    fps                  | 76           |
|    iterations           | 2            |
|    time_elapsed         | 53           |
|    total_timesteps      | 24576        |
| train/                  |              |
|    approx_kl            | 0.0055058803 |
|    clip_fraction        | 0.0531       |
|    clip_range           | 0.2          |
|    entropy_loss         | -12.8        |
|    explained_variance   | 0.00565      |
|    learning_rate        | 0.0003       |
|    loss                 | 5.1          |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00376     |
|    std                  | 1            |
|    value_loss           | 10.3         |
------------------------------------------
-----------------------------------------
| rollout/  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 11.5        |
| time/                   |             |
|    fps                  | 76          |
|    iterations           | 3           |
|    time_elapsed         | 80          |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.008373048 |
|    clip_fraction        | 0.0832      |
|    clip_range           | 0.2         |
|    entropy_loss         | -12.6       |
|    explained_variance   | 0.00346     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.14        |
|    n_updates            | 220         |
|    policy_gradient_loss | -0.00542    |
|    std                  | 0.98        |
|    value_loss           | 10.3        |
-----------------------------------------
------------------------------------------
| rollout/                |      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 11.5        |
| time/                   |             |
|    fps                  | 75          |
|    iterations           | 4           |
|    time_elapsed         | 107         |
|    total_timesteps      | 69632       |
| train/                  |             |
|    approx_kl            | 0.009138284 |
|    clip_fraction        | 0.119       |
|    clip_range           | 0.2         |
|    entropy_loss         | -12.4       |
|    explained_variance   | 0.0036      |
|    learning_rate        | 0.0003      |
|    loss                 | 4.48        |
|    n_updates            | 330         |
|    policy_gradient_loss | -0.00728    |
|    std                  | 0.959       |
|    value_loss           | 10.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 11.6        |
| time/                   |             |
|    fps                  | 75          |
|    iterations           | 5           |
|    time_elapsed         | 134         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.008274239 |
|    clip_fraction        | 0.0962      |
|    clip_range           | 0.2         |
|    entropy_loss         | -12.3       |
|    explained_variance   | 0.0024      |
|    learning_rate        | 0.0003      |
|    loss                 | 4.71        |
|    n_updates            | 440         |
|    policy_gradient_loss | -0.00611    |
|    std                  | 0.952       |
|    value_loss           | 10.5        |
-----------------------------------------
---------------------------------
| rollout/           |          |
|    ep_

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 11.7        |
| time/                   |             |
|    fps                  | 72          |
|    iterations           | 5           |
|    time_elapsed         | 141         |
|    total_timesteps      | 204800      |
| train/                  |             |
|    approx_kl            | 0.013273247 |
|    clip_fraction        | 0.142       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.6       |
|    explained_variance   | 0.00382     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.14        |
|    n_updates            | 990         |
|    policy_gradient_loss | -0.00941    |
|    std                  | 0.883       |
|    value_loss           | 10.3        |
-----------------------------------------
---------------------------------
| rollout/           |          |
|    ep_

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12       |
|    ep_rew_mean     | 11.5     |
| time/              |          |
|    fps             | 75       |
|    iterations      | 1        |
|    time_elapsed    | 27       |
|    total_timesteps | 227328   |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 11.6        |
| time/                   |             |
|    fps                  | 74          |
|    iterations           | 2           |
|    time_elapsed         | 54          |
|    total_timesteps      | 229376      |
| train/                  |             |
|    approx_kl            | 0.009522015 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.6       |
|    explained_variance   | 0.00431     |
|    learning_rate        | 0.

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 12           |
|    ep_rew_mean          | 11.5         |
| time/                   |              |
|    fps                  | 76           |
|    iterations           | 2            |
|    time_elapsed         | 53           |
|    total_timesteps      | 249856       |
| train/                  |              |
|    approx_kl            | 0.0057407534 |
|    clip_fraction        | 0.0807       |
|    clip_range           | 0.2          |
|    entropy_loss         | -11.5        |
|    explained_variance   | 0.0052       |
|    learning_rate        | 0.0003       |
|    loss                 | 5.33         |
|    n_updates            | 1210         |
|    policy_gradient_loss | -0.00376     |
|    std                  | 0.876        |
|    value_loss           | 10.3         |
------------------------------------------
-----------------------------------------
| rollout/  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 11.5        |
| time/                   |             |
|    fps                  | 76          |
|    iterations           | 3           |
|    time_elapsed         | 80          |
|    total_timesteps      | 272384      |
| train/                  |             |
|    approx_kl            | 0.009715384 |
|    clip_fraction        | 0.13        |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.4       |
|    explained_variance   | 0.00617     |
|    learning_rate        | 0.0003      |
|    loss                 | 4.65        |
|    n_updates            | 1320        |
|    policy_gradient_loss | -0.00779    |
|    std                  | 0.864       |
|    value_loss           | 10.2        |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | 11.4        |
| time/                   |             |
|    fps                  | 76          |
|    iterations           | 4           |
|    time_elapsed         | 107         |
|    total_timesteps      | 294912      |
| train/                  |             |
|    approx_kl            | 0.010567042 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.4       |
|    explained_variance   | 0.00631     |
|    learning_rate        | 0.0003      |
|    loss                 | 4.96        |
|    n_updates            | 1430        |
|    policy_gradient_loss | -0.00541    |
|    std                  | 0.865       |
|    value_loss           | 10.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |       

In [ ]:
for _ in range(10):
    obs = env.reset()
    done = False
    while not done:
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())